In [1]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

column_renaming_dict = {'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'}

A new version of galpy (1.10.0) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [2]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
#dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
dfv = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/mustache_work/figures/cubes/edenhofer_xyz_4pc.csv')
dfv = dfv.loc[dfv['e'].between(0.003, 1)]
#dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 1
scatter_edenhofer = go.Scatter3d(
    x=dfv['x'].values[::ds_index],
    y=dfv['y'].values[::ds_index],
    z=dfv['z'].values[::ds_index],
    mode='markers',
    marker=dict(size=2,
                color='gray',
                symbol='circle',
                opacity=.02),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0

# gsh238 shell
dfv = pd.read_csv('/Users/cam/Downloads/gsh_238_region_edenhofer.csv')
dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
scatter_gsh238 = go.Scatter3d(
    x=dfv['X'].values,
    y=dfv['Y'].values,
    z=dfv['Z'].values,
    mode='markers',
    marker=dict(size=1.5,
                color='gray',
                symbol='circle',
                opacity=.2),
    line = dict(color = 'gray', width = 0.),
    name='GSH238',
    visible = 'legendonly',
    hovertext='GSH238')
# static_traces.append(scatter_gsh238)
static_traces_times.append([0.]) # only show at t=0

# read in leike map

hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
# static_traces.append(scatter_lb)
# static_traces_times.append([0.]) # only show at t=0

# annie IVS 
ivs = pd.read_csv('/Users/cam/Downloads/npix128boundary_mean_sigma10.csv')
ivs_out = pd.read_csv('/Users/cam/Downloads/npix128boundaryout_mean_sigma10.csv')
ivs = pd.concat([ivs, ivs_out])
ivs = ivs.sample(50000)
scatter_ivs = go.Scatter3d(
    x = ivs['x[pc]'].values,
    y = ivs['y[pc]'].values,
    z = ivs['z[pc]'].values,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='IVS (Annie)',
    visible = 'legendonly',
    hovertext='IVS')
# static_traces.append(scatter_ivs)
# scatter_ivs_times = np.arange(0, -5., -0.5)
# static_traces_times.append(scatter_ivs_times)

In [3]:
df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_catalog_oct_13.csv')
#df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_sample_chronos_ages.csv')
df_hunt_full = df_hunt_full.rename(columns={'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'})

df_hunt_good = df_hunt_full.loc[
    (df_hunt_full['n_rv_gaia_apogee_galah'] >= 3) & 
    (df_hunt_full['U_err'] < 10) & 
    (df_hunt_full['V_err'] < 10) & 
    (df_hunt_full['W_err'] < 10)
    ]
df_hunt_young = df_hunt_good.loc[df_hunt_good['age_myr'] < 10]
df_hunt_older = df_hunt_good.loc[df_hunt_good['age_myr'].between(0, 150)]
df_hunt_older.to_csv('/Users/cam/Downloads/hunt_catalog_0_70_myr.csv', index=False)

In [4]:
fam_list = [
    'HSC_1340',
    'CWNU_1111',
    'NGC_2232'
    'CWNU_1057',
    'HSC_1481',
    'ZHBJZ_1',
    'HSC_1373',
    'HSC_1460',
    'OCSN_50',
    'ASCC_24',
    'Theia_71'
]
a = df_hunt_good.loc[df_hunt_good['name'].isin(fam_list)]

In [5]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')

# CHRONOS ages
df_hunt_new_ages = pd.read_csv('/Users/cam/Downloads/hunt_sample_chronos_ages_2.csv')
df_hunt_gagne = df_hunt_gagne.merge(df_hunt_new_ages, on='name', how='left', suffixes=('', '_new_ages'))

# use 'age_chronos_mode' for age if it exists, otherwise use 'age_myr'
df_hunt_gagne['age'] = df_hunt_gagne['age_chronos_mode']
df_hunt_gagne.loc[df_hunt_gagne['age'].isnull(), 'age'] = df_hunt_gagne['age_myr']
df_hunt_gagne['age_myr'] = df_hunt_gagne['age']
df_hunt_gagne.to_csv('/Users/cam/Downloads/cluster_sample_data_chornos_ages.csv', index=False)

df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']
gamma_vel = df_hunt_gagne[df_hunt_gagne['family'] == 'gamma_velorum']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [27.], 'n_stars' : [1]})


In [6]:
df_cr135_ivs = df_hunt_gagne.loc[df_hunt_gagne['name'].isin(['Alessi_36', 'Collinder_135', 'Collinder_140', 'NGC_2451B', 'OCSN_82'])]
print(df_cr135_ivs['age_myr'].median())

df_gammaVel_ivs = df_hunt_gagne.loc[df_hunt_gagne['name'].isin(['Pozzo_1', 'CWNU_1083', 'CWNU_1096', 'OC_0470', 'OC_0479'])]
print(df_gammaVel_ivs['age_myr'].min())
print(df_gammaVel_ivs['age_myr'].max())


35.98241640625828
9.313057268062932
12.206314418559057


In [7]:
# RW/Split/CS clusters
rw_region = pd.read_csv('/Users/cam/Downloads/rw_clusters.csv')
rw = rw_region.loc[rw_region.age_myr < 15]
remove = ['Theia_6', 'Teutsch_39', 'OCSN_32', 'Aveni-Hunter_1', 'IC_5146', 'Collinder_419', 'Dolidze_8', 'Theia_1920', 'Collinder_95', 'Mon_OB1-D', 'NGC_2264', 'UBC_159']
rw = rw[~rw['name'].isin(remove)]
rw = rw.rename(columns = column_renaming_dict)
#lacerta = ['HSC_661', 'vdBergh_152', 'HSC_705', 'OCSN_32', 'UBC_159', 'UPK_168', 'UPK_166', 'Teutch_39', 'Theia_100']

# The Split
split_region = pd.read_csv('/Users/cam/Downloads/the_split_selection.csv')
split = split_region.loc[split_region.age_myr < 15]
split = split.rename(columns = column_renaming_dict)

# Cepheus spur
cepheus_region = pd.read_csv('/Users/cam/Downloads/cepheus_spur_region.csv')
cepheus_spur = cepheus_region.loc[cepheus_region.age_myr < 30]
cepheus_spur = cepheus_spur.rename(columns = column_renaming_dict)

# new spur?
spur_clusters = ['Pismis_5', 'Collinder_197', 'FSR_1421', 'BH_56', 'HSC_2144', 'ASCC_79', 'UPK_604', 'FSR_1694', 'NGC_6178', 'OC_0684', 'HSC_2911', 'CWNU_170', 'NGC_6383']
spur = df_hunt_full.loc[df_hunt_full['name'].isin(spur_clusters)]
spur = spur.rename(columns = column_renaming_dict)

In [8]:
# omuamua
x,y,z = 0., 0., 27.
U, V, W = -11.622, -22.461, -7.824
# U, V, W = 33.1, -6.8, 8.3
name = 'Oumuamua'
age = 1000.

omuamua = pd.DataFrame({'name' : [name], 'family' : ['Oumuamua'], 'age_myr' : [age], 'U' : [U], 'V' : [V], 'W' : [W], 'x' : [x], 'y' : [y], 'z' : [z], 'n_stars' : [1]})

In [9]:
# new families
new_fam_1 = ['HSC_1373', 'OCSN_50', 'ASCC_24', 'HSC_1460', 'Theia_71', 'HSC_1340', 'CWNU_1057', 'CWNU_1111', 'CWNU_1092', 'HSC_1481', 'HSC_921', 'NGC_2232']
df_new_fam_1 = df_hunt_full.loc[df_hunt_full['name'].isin(new_fam_1)]

lacerta = ['HSC_661', 'vdBergh_152', 'HSC_705', 'OCSN_32', 'UBC_159', 'UPK_168', 'UPK_166', 'Teutch_39', 'Theia_100', 'HSC_699', 'CWNU_311', 'Teutsch_39', 'Aveni-Hunter_1', 'HSC_886']
df_lacerta = df_hunt_full.loc[df_hunt_full['name'].isin(lacerta)]

alphaPer_interloper = ['CWNU_1183', 'HSC_633', 'HSC_693', 'HSC_674', 'RSG_5', 'Theia_96', 'HSC_782', 'CWNU_519']
df_ap_interloper= df_hunt_full.loc[df_hunt_full['name'].isin(alphaPer_interloper)]


In [10]:
df_mustache = pd.read_csv('/Users/cam/Downloads/mustache_with_3d_vels_connected_new.csv')
df_mustache = df_mustache.sample(frac=.25)
df_mustache = df_mustache.rename(columns={'X' : 'x', 'Y' : 'y', 'Z' : 'z'})
df_mustache['age_myr'] = 30.
df_mustache['name'] = 'mustache_a'

In [11]:
mustache = StarClusterData(df_mustache, data_name = 'The Mustache', min_size = 0, max_size = 10, color = 'cyan', opacity = 0.1, marker_style = 'circle')

# Dark plot

In [12]:
# spurs
rw_data = StarClusterData(rw, data_name = 'Radcliffe Wave', min_size = 1, max_size = 12, color = 'red', opacity = 1., marker_style = 'circle', show_tracks = False)
split_data = StarClusterData(split, data_name = 'The Split', min_size = 1, max_size = 12, color = '#FFA07A', opacity = 1., marker_style = 'circle', show_tracks = False)
cepheus_spur_data = StarClusterData(cepheus_spur, data_name = 'Cepheus Spur', min_size = 1, max_size = 12, color = 'cyan', opacity = 1., marker_style = 'circle', show_tracks = False)
new_spur = StarClusterData(spur, data_name = 'Sag-Vela Spur (new?)', min_size = 1, max_size = 12, color = '#90EE90', opacity = 1., marker_style = 'circle', show_tracks = False)

# families
sample_data = StarClusterData(df_hunt_gagne, data_name = 'Clean Clusters Sample', min_size = 0, max_size = 9, color = 'gray', opacity = 1., marker_style = 'circle')
ap_data = StarClusterData(ap, data_name = '⍺Per', min_size = 0, max_size = 9, color = 'violet', opacity = 1., marker_style = 'circle')
m6_data = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 9, color = 'cyan', opacity = 1., marker_style = 'circle')
cr135_data = StarClusterData(cr135, data_name = 'Cr135', min_size = 0, max_size = 9, color = 'orange', opacity = 1., marker_style = 'circle')
gamma_vel_data = StarClusterData(gamma_vel, data_name = '𝝲Vel', min_size = 0, max_size = 9, color = 'lime', opacity = 1., marker_style = 'circle')

# new families
unnamed_family_1= StarClusterData(df_new_fam_1, data_name = 'Unnamed family 1', min_size = 0, max_size = 7, color = 'gray', opacity = 1., marker_style = 'circle')
lacerta = StarClusterData(df_lacerta, data_name = 'Lacerta', min_size = 0, max_size = 7, color = 'gray', opacity = 1., marker_style = 'circle')
ap_interloper = StarClusterData(df_ap_interloper, data_name = '⍺Per interloper', min_size = 0, max_size = 7, color = 'gray', opacity = 1., marker_style = 'circle')

young_data = StarClusterData(df_hunt_young, data_name = 'Clusters (< 10 Myr)', min_size = 0., max_size = 9, color = 'white', opacity = 1., marker_style = 'circle')
older_data = StarClusterData(df_hunt_older, data_name = 'Clusters (< 70 Myr)', min_size = 0., max_size = 3, color = 'gray', opacity = 1., marker_style = 'circle')
sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 8, color = 'yellow', opacity = 1., marker_style = 'circle')

#omuamua
omuamua_data = StarClusterData(omuamua, data_name = 'Oumuamua', min_size = 0, max_size = 20., color = 'red', opacity = 1., marker_style = 'circle')

# mate data
df_mate = pd.read_csv('/Users/cam/Downloads/lacerta_members_3dvelocities (1).csv')
df_mate = df_mate.rename(columns={'label' : 'name', 'x_helioc' : 'x', 'y_helioc' : 'y', 'z_helioc' : 'z', 'u_hel' : 'U', 'v_hel' : 'V', 'w_hel' : 'W'})
df_mate = df_mate.groupby('name').median().reset_index()
lacerta = StarClusterData(df_mate, data_name = 'Lacerta', min_size = 0, max_size = 9, color = 'cyan', opacity = 1., marker_style = 'circle')

sc = StarClusterCollection([sample_data, young_data, older_data, sun_data, ap_data, m6_data, cr135_data, gamma_vel_data, rw_data, split_data, cepheus_spur_data, new_spur, unnamed_family_1, lacerta, ap_interloper, mustache, omuamua_data])
grouping_dict = {
    'Cluster families' : ['Clean Clusters Sample', '⍺Per', 'M6', 'Cr135', '𝝲Vel', 'Sun'],
    'Dust structures' : ['Young clusters', 'Radcliffe Wave', 'The Split', 'Cepheus Spur', 'Sag-Vela Spur (new?)', 'Sun'],
    'The Mustache and LB clusters' : ['The Mustache', '⍺Per', 'Sun'],
    'Omuamua with Alpha Per' : ['Oumuamua', '⍺Per', 'Sun']
}


# ap_data.opacity = .5
# m6_data.opacity = .5
# cr135_data.opacity = .5
# ap_data.color = 'cyan'
# m6_data.color = 'cyan'
# cr135_data.color = 'cyan'
# rw_data.color = 'red'
# split_data.color = 'red'
# new_spur.color = 'red'
# cepheus_spur_data.color = 'red'

# grouping_dict = {
#     'Dust structures' : ['Radcliffe Wave', 'The Split', 'Cepheus Spur', 'Sag-Vela Spur (new?)', 'Sun'],
#     'Dust structures and cluster families' : ['Radcliffe Wave', 'The Split', 'Cepheus Spur', 'Sag-Vela Spur (new?)', '⍺Per', 'M6', 'Cr135', 'Sun']
# }
#grouping_dict = None

In [ ]:
xyz_widths = (1200, 1200, 500)

for c in sc.get_all_clusters():
    if c.data_name not in ['The Mustache', 'Oumuamua', 'Sun'] and 'n_stars' in c.df.columns:
        c.size_by_n_stars = True
        c.max_size = 20
        c.min_size = 3
    # if c.data_name in ['Cr135', 'M6', '⍺Per']:
    #     c.show_tracks = True

plot_3d = StarClusters3DPlotter(
    data_collection = sc, 
    xyz_widths = xyz_widths, 
    figure_theme = 'dark', 
    trace_grouping_dict=grouping_dict)

save_name = '/Users/cam/Downloads/gal_movie.html'
time_int = np.round(np.arange(0., -60.5, -0.5), 1)

fig3d = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = static_traces, 
    static_traces_times = static_traces_times, 
    static_traces_legendonly=False,
    focus_group = None,
    fade_in_time = 6, # Myr,
    fade_in_and_out = True
)